<a href="https://colab.research.google.com/github/sagarnibandhe/MachineLearning/blob/master/mobileApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import pandas as pd
import numpy as np
import sklearn

In [31]:
titanic_data = pd.read_csv('titanic.csv')
titanic_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [32]:
import warnings
warnings.filterwarnings("ignore")

In [33]:
titanic_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [34]:
titanic_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [35]:
titanic_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [36]:
titanic_data.drop("Cabin", axis=1,inplace=True)

In [37]:
titanic_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
dtype: int64

In [38]:
#fill the missing values in age column
titanic_data["Age"].fillna(titanic_data["Age"].mean(), inplace=True)

In [39]:
titanic_data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       2
dtype: int64

In [40]:
titanic_data.dropna(inplace=True)

In [41]:
titanic_data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [42]:
#create dummies: to convert to numerics
sex = pd.get_dummies(titanic_data['Sex'],drop_first=True)
sex.head()

,male
0,1
1,0
2,0
3,0
4,1


In [43]:
Pcl = pd.get_dummies(titanic_data['Pclass'],drop_first=True)
Pcl.head()

,2,3
0,0,1
1,0,0
2,0,1
3,0,0
4,0,1


In [44]:
embark = pd.get_dummies(titanic_data["Embarked"],drop_first=True)
embark.head(2)

,Q,S
0,0,1
1,0,0


In [45]:
titanic_data = pd.concat([titanic_data, sex,Pcl,embark], axis=1)
titanic_data.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,male,2,3,Q,S
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,1,0,1,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,0,0,0,0,0


In [46]:
titanic_data.drop(['Sex', 'Embarked', 'Pclass', 'Name', 'Ticket','PassengerId'],axis=1, inplace=True)
titanic_data.head()

,Survived,Age,SibSp,Parch,Fare,male,2,3,Q,S
0,0,22.0,1,0,7.2500,1,0,1,0,1
1,1,38.0,1,0,71.2833,0,0,0,0,0
2,1,26.0,0,0,7.9250,0,0,1,0,1
3,1,35.0,1,0,53.1000,0,0,0,0,1
4,0,35.0,0,0,8.0500,1,0,1,0,1


In [47]:
X = titanic_data.drop('Survived', axis=1)
y = titanic_data['Survived']

from sklearn.model_selection import train_test_split

In [48]:
X_train, X_test, y_train,y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [49]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [50]:
predictions = logmodel.predict(X_test)

In [51]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, predictions)*100)

83.14606741573034


In [52]:
predictions = logmodel.predict([[22.0,1,0,7.2500,1,0,1,0,1]])
predictions

array([0])

In [53]:
!pip install flask-ngrok

In [54]:
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify

app = Flask(__name__)
run_with_ngrok(app) #starts ngrok when the app is running
@app.route("/<int:Age>/<int:SibSp>/<int:Parch>/<float:Fare>/<Gender>/<int:PClass>/<Place>")
def home(Age,SibSp,Parch,Fare,Gender,PClass,Place):
  p = []
  p += [Age,SibSp,Parch,Fare]
  if Gender.casefold() == "m":
    p +=[1]
  else:
    p +=0
  if PClass == 2:
    p += [1,0]
  elif PClass == 3:
    p +=[0,1]
  else:
    p +=[0,0]
  if Place.casefold == "queenstown":
    p +=[1,0]
  elif Place.casefold == "southampton":
    p +=[0,1]
  else:
    p +=[0,0]
  arr = np.array([p])
  predict = logmodel.predict(arr)
  if predict ==[1]:
    result = {'result':'Survived'}
  else:
    result = {'result':'NotSurvived'}
    
  return jsonify(result)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://2af4d9b06500.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [03/Dec/2020 02:35:58] "GET //22/1/0/7.25/M/3/southampton HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2020 02:39:04] "GET //22/1/0/7.25/M/3/ HTTP/1.1" 404 -
127.0.0.1 - - [03/Dec/2020 02:39:12] "GET //22/1/0/7.25/M/3/southampton HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2020 02:45:01] "GET //22/1/0/7.25/M/3 HTTP/1.1" 404 -
